In [1]:
import time
import fastf1
import logging
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

from concurrent.futures import ThreadPoolExecutor, as_completed

fastf1.Cache.enable_cache('./.cache')  # replace with your cache directory
logging.getLogger('fastf1').setLevel(logging.ERROR)  # This will disable both logs and warnings
#fastf1.Cache.offline_mode(True)

def get_lap_data(session):
    """
    Get lap data for a given session.
    Args:
        session (fastf1.core.Session): The session object from FastF1.
    Returns:
        pd.DataFrame: A DataFrame containing lap data with relevant columns.
    """
    lap = session.laps
    lap_prac = lap[(lap['IsAccurate']==True) & (lap['IsPersonalBest']==True)]
    lap_prac = lap_prac.groupby('Driver').apply(
        lambda x: x[x['LapTime'] == x['LapTime'].min()]).sort_values('LapTime').reset_index(drop=True)[['Driver','Team']]
    for col in lap_prac.select_dtypes(include='timedelta64[ns]').columns:
        lap_prac[col] = lap_prac[col].dt.total_seconds()
    lap_prac.reset_index(inplace=True)
    lap_prac.rename(columns={'index':'Position'},inplace=True)
    lap_prac['Position'] = lap_prac['Position'] + 1
    lap_prac['session'] = session.name
    lap_prac['country'] = session.event.EventName
    lap_prac['year'] = session.event.year
    lap_prac['Date'] = session.event.EventDate
    
    return lap_prac

def fetch_lap_data(year, race, race_type):
    try:
        session = fastf1.get_session(year, race, race_type)
        session.load()
        time.sleep(5)  # To avoid overwhelming the server

        return get_lap_data(session)
    except Exception as e:
        print(f"Could not load session for year: {year} and race {race} due to {e}")
        return None

qualifying_sessions = pd.DataFrame([])
race_type = 'Q'
tasks = [(year, race, race_type) for race in range(1, 25) for year in [2022, 2023, 2024, 2025]]

with ThreadPoolExecutor(max_workers=4) as executor:
    futures = [executor.submit(fetch_lap_data, year, race, race_type) for year, race, race_type in tasks]
    for future in tqdm(as_completed(futures), total=len(futures)):
        result = future.result()
        if result is not None:
            qualifying_sessions = pd.concat([qualifying_sessions, result], ignore_index=True)

current_drivers = qualifying_sessions[qualifying_sessions['year']==2025].Driver.unique().tolist()
print(f"Current Drivers: {current_drivers}")
circuits = qualifying_sessions['country'].unique().tolist()
print(f"Circuits: {circuits}")
team_mapping = dict(zip(sorted(qualifying_sessions.Team.value_counts().keys().tolist()),[1,2,3,4,5,1,6,7,8,2,9,10,11]))
print(f"Team Mapping: {team_mapping}")

driver_stats_list = []

for i in current_drivers:

    driver_sessions = qualifying_sessions[qualifying_sessions['Driver'] == i]
    recent_sessions = driver_sessions[driver_sessions['year'] == 2025]
    avg_driver_pos = driver_sessions.groupby('country')['Position'].mean().sort_values().to_dict()
    team_name = driver_sessions['Team'].values[0]
    team_perf = qualifying_sessions[qualifying_sessions['Team'] == team_name].groupby('country')['Position'].mean().sort_values().to_dict()
    
    try : 
        lp = int(recent_sessions[recent_sessions['country'] == con]['Position'].iloc[0])
    except:
        lp = -1
    
    try:
        tf = float(team_perf[con])
    except:
        tf = -1
    
    if 'Great Britain' in avg_driver_pos.keys() and 'United Kingdom' in avg_driver_pos.keys():
        avg_driver_pos['Great Britain'] = (avg_driver_pos['United Kingdom'] + avg_driver_pos['Great Britain'])/2
        avg_driver_pos.pop('United Kingdom')
    elif 'United Kingdom' in avg_driver_pos.keys() and  'Great Britain' not in avg_driver_pos.keys():
        avg_driver_pos['Great Britain'] = avg_driver_pos['United Kingdom']
        avg_driver_pos.pop('United Kingdom')
    
    for con in avg_driver_pos.keys():
        driver_stats_list.append({
            'Name': i,
            'Avg_Position_Past': float(driver_sessions['Position'].mean()),
            'Best_Position_Past': int(driver_sessions['Position'].min()),
            'Worst_Position_Past': int(driver_sessions['Position'].max()),
            'Best_Position_Recent': int(recent_sessions['Position'].min()),
            'Worst_Position_Recent': int(recent_sessions['Position'].max()),
            'Avg_Position_Recent': float(recent_sessions['Position'].mean()),
            'Last_Position': lp,
            'Avg_Finish_in_this_circuit' : float(avg_driver_pos[con]),
            'Avg_Team_Finish_in_this_circuit' : tf,
            'Country': con,
            'Team': team_mapping[team_name]
        })

pd.DataFrame(driver_stats_list).to_csv('.cache/hist_data/team_driver_performance.csv')


 75%|███████████████████████████████████████████████████████████████████████████                         | 72/96 [04:34<01:33,  3.91s/it]

Could not load session for year: 2025 and race 18 due to The data you are trying to access has not been loaded yet. See `Session.load`


 82%|██████████████████████████████████████████████████████████████████████████████████▎                 | 79/96 [04:52<00:43,  2.56s/it]

Could not load session for year: 2025 and race 19 due to The data you are trying to access has not been loaded yet. See `Session.load`


 86%|██████████████████████████████████████████████████████████████████████████████████████▍             | 83/96 [05:09<00:35,  2.72s/it]

...

Could not load session for year: 2025 and race 22 due to The data you are trying to access has not been loaded yet. See `Session.load`


 99%|██████████████████████████████████████████████████████████████████████████████████████████████████▉ | 95/96 [05:35<00:02,  2.32s/it]

Could not load session for year: 2025 and race 23 due to The data you are trying to access has not been loaded yet. See `Session.load`


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 96/96 [05:40<00:00,  3.55s/it]

Could not load session for year: 2025 and race 24 due to The data you are trying to access has not been loaded yet. See `Session.load`
Current Drivers: ['NOR', 'PIA', 'VER', 'RUS', 'TSU', 'ALB', 'LEC', 'HAM', 'SAI', 'GAS', 'HAD', 'ALO', 'DOO', 'STR', 'BOR', 'ANT', 'HUL', 'LAW', 'OCO', 'BEA', 'COL']
Circuits: ['Bahrain Grand Prix', 'Australian Grand Prix', 'Chinese Grand Prix', 'Saudi Arabian Grand Prix', 'Japanese Grand Prix', 'Azerbaijan Grand Prix', 'Emilia Romagna Grand Prix', 'Miami Grand Prix', 'Spanish Grand Prix', 'Monaco Grand Prix', 'Canadian Grand Prix', 'Austrian Grand Prix', 'British Grand Prix', 'Hungarian Grand Prix', 'French Grand Prix', 'Belgian Grand Prix', 'Dutch Grand Prix', 'Italian Grand Prix', 'Singapore Grand Prix', 'Qatar Grand Prix', 'United States Grand Prix', 'Mexico City Grand Prix', 'São Paulo Grand Prix', 'Las Vegas Grand Prix', 'Abu Dhabi Grand Prix']
Team Mapping: {'Alfa Romeo': 1, 'AlphaTauri': 2, 'Alpine': 3, 'Aston Martin': 4, 'Ferrari': 5, 'Haas F1 T